In [133]:
import numpy as np
import pandas as pd 
from statsmodels.tsa.stattools import acf, pacf, acovf
import warnings
from scipy import stats


In [201]:
def DMtest(e1,e2,h=1,power=2,alternative='two.sided'):
    d = np.abs(e1)**power - np.abs(e2)**2
    dcov = acovf(d,nlag=h-1,fft=False,missing='drop')
    dvar = np.sum(np.append(dcov[0],2*dcov[1:]))/len(d)
    dv = dvar
    if dv>0:
        STATISTIC =np.nanmean(d)/np.sqrt(dv)
    elif (h==1):
        raise ValueError("Variance of DM statistic is zero")
    else: 
        warnings.warn('Variance is negative, using horizon h=1') 
        DMtest(e1,e2,h=1,power=power)   
    n = len(d)
    k = ((n + 1 - 2 * h + (h / n) * (h - 1)) / n) **(0.5)
    STATISTIC = STATISTIC *k 
    
    if alternative == 'two.sided':
        PVAL = 2*stats.t.cdf(-np.abs(STATISTIC),df=n-1)
    elif alternative == 'less':
        PVAL = stats.t.cdf(STATISTIC,df=n-1)
    elif alternative == 'greater':
        PVAL = 1-stats.t.cdf(STATISTIC,df=n-1)
        
    PARAMETER = [h,power]
    return_DICT = {"DM":STATISTIC,'forecast_horizon':h,'power':power,'p_val':PVAL,'Method':'Diebold-Mariano Test'
                  }
    return return_DICT


def bh_correction(pval,level=0.05):
    nn = len(pval)
    pv_srt = sorted(pval)
    limits = np.arange(1,nn+1)*level/nn
    
    if len(np.where(pv_srt<limits))==0:
        discoveries = 0
        return discoveries
    else:
        discoveries = np.max(np.where(pv_srt<limits))
        return discoveries/nn 

In [171]:
DF = pd.read_csv('dcov.csv')
d=np.array(DF.loc[0,:])
e1 = pd.read_csv('TESTwillfout1.csv')
e1=np.array(e1.loc[0,:])

e2 = pd.read_csv('TESTwillfout2.csv')
e2=np.array(e2.loc[0,:])
DMtest(e1,e2,h=2,power=2,alternative='two.sided')

{'DM': 0.03609604786918916,
 'forecast_horizon': 2,
 'power': 2,
 'p_val': 0.9715823158790085,
 'Method': 'Diebold-Mariano Test'}